# TMBD Data API
----


In [26]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import os
import locale # to format currency as USD
from pandas.io.json import json_normalize
locale.setlocale( locale.LC_ALL, '' )

# Import API key
from api_keys import api_key

# Output File (CSV)
output_data_file = "tmbddata_dev.csv"
movie_revenue_data = "tmbd_dev_movie_revenue.csv"
movie_genres_data = "tmbd_dev_genres.csv"
        

In [2]:
## Generate Movies Database from TMBD API by looping though all pages
all_results = []
url = 'https://api.themoviedb.org/3/discover/movie/'
page = 1
while True:
    r = requests.get(url + '?api_key='+ api_key+'&language=en-US' + '&page='+str(page))
    movie_data = r.json()
    if r.status_code == 200:
        all_results.append(movie_data)
        page += 1
    else:
        break

In [3]:
#Create dataframe

In [15]:
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(all_results[:20])

[{'page': 1,
  'results': [{'adult': False,
               'backdrop_path': '/p3TCqUDoVsrIm8fHK9KOTfWnDjZ.jpg',
               'genre_ids': [12, 18, 9648, 878, 53],
               'id': 419704,
               'original_language': 'en',
               'original_title': 'Ad Astra',
               'overview': 'The near future, a time when both hope and '
                           'hardships drive humanity to look to the stars and '
                           'beyond. While a mysterious phenomenon menaces to '
                           'destroy life on planet Earth, astronaut Roy '
                           'McBride undertakes a mission across the immensity '
                           'of space and its many perils to uncover the truth '
                           'about a lost expedition that decades before boldly '
                           'faced emptiness and silence in search of the '
                           'unknown.',
               'popularity': 469.061,
               'post

               'vote_count': 10723},
              {'adult': False,
               'backdrop_path': '/OIGX2lm5tmlCKvZUghtwHzoxxO.jpg',
               'genre_ids': [28, 35, 14],
               'id': 287947,
               'original_language': 'en',
               'original_title': 'Shazam!',
               'overview': 'A boy is given the ability to become an adult '
                           'superhero in times of need with a single magic '
                           'word.',
               'popularity': 34.281,
               'poster_path': '/xnopI5Xtky18MPhK40cZAGAOVeV.jpg',
               'release_date': '2019-03-29',
               'title': 'Shazam!',
               'video': False,
               'vote_average': 7,
               'vote_count': 4038},
              {'adult': False,
               'backdrop_path': '/c2Ax8Rox5g6CneChwy1gmu4UbSb.jpg',
               'genre_ids': [28, 12, 14, 878],
               'id': 140607,
               'original_language': 'en',
               'or

               'genre_ids': [18, 36],
               'id': 45269,
               'original_language': 'en',
               'original_title': "The King's Speech",
               'overview': "The King's Speech tells the story of the man who "
                           'became King George VI, the father of Queen '
                           'Elizabeth II. After his brother abdicates, George '
                           "('Bertie') reluctantly assumes the throne. Plagued "
                           'by a dreaded stutter and considered unfit to be '
                           'king, Bertie engages the help of an unorthodox '
                           'speech therapist named Lionel Logue. Through a set '
                           'of unexpected techniques, and as a result of an '
                           'unlikely friendship, Bertie is able to find his '
                           'voice and boldly lead the country into war.',
               'popularity': 13.693,
               'poster

                           "the mysterious power to heal people's ailments. "
                           "When the cell block's head guard, Paul Edgecomb, "
                           "recognizes Coffey's miraculous gift, he tries "
                           "desperately to help stave off the condemned man's "
                           'execution.',
               'popularity': 26.266,
               'poster_path': '/sOHqdY1RnSn6kcfAHKu28jvTebE.jpg',
               'release_date': '1999-12-10',
               'title': 'The Green Mile',
               'video': False,
               'vote_average': 8.5,
               'vote_count': 9253},
              {'adult': False,
               'backdrop_path': '/rm2oMykm5nX6SzXFr7TGHkO6r8Z.jpg',
               'genre_ids': [12, 14],
               'id': 428078,
               'original_language': 'en',
               'original_title': 'Mortal Engines',
               'overview': 'Many thousands of years in the future, Earth’s '
                 

               'original_language': 'ja',
               'original_title': 'ルパンレンジャーＶＳパトレンジャーＶＳキュウレンジャー',
               'overview': '"Lupinranger VS Patranger VS Kyuranger" is an '
                           'upcoming V-Cinext film between Kaitou Sentai '
                           'Lupinranger VS Keisatsu Sentai Patranger and Uchu '
                           'Sentai Kyuranger. The story begins when the '
                           'Lupinrangers, Kairi, Touma, and Umika, are '
                           'kidnapped by someone mysterious. The Patrangers '
                           'are then tasked with an Abduction Case to find the '
                           'missing thieves, where they run into the Kyuranger '
                           'team as they pass through space. Just who exactly '
                           'kidnapped them? And why did the 12 Kyurangers '
                           'return to space?',
               'popularity': 23.645,
               'poster_path': '/dP7c

               'overview': 'Skipper, Kowalski, Rico and Private join forces '
                           'with undercover organization The North Wind to '
                           'stop the villainous Dr. Octavius Brine from '
                           'destroying the world as we know it.',
               'popularity': 20.086,
               'poster_path': '/pRer59eyY4rjkLPeKXKo7fPxCHH.jpg',
               'release_date': '2014-11-22',
               'title': 'Penguins of Madagascar',
               'video': False,
               'vote_average': 6.4,
               'vote_count': 2476},
              {'adult': False,
               'backdrop_path': '/1iV72CbpJihklJ5pIT7mUYOoy6O.jpg',
               'genre_ids': [27],
               'id': 655257,
               'original_language': 'id',
               'original_title': 'Surat dari Kematian',
               'overview': 'Plot unknown.',
               'popularity': 15.522,
               'poster_path': '/bpB0NVeriC7bR0hGB74DdbMsIb4.jpg

               'genre_ids': [28, 53],
               'id': 562,
               'original_language': 'en',
               'original_title': 'Die Hard',
               'overview': "NYPD cop John McClane's plan to reconcile with his "
                           'estranged wife is thrown for a serious loop when, '
                           'minutes after he arrives at her office, the entire '
                           'building is overtaken by a group of terrorists. '
                           'With little help from the LAPD, wisecracking '
                           'McClane sets out to single-handedly rescue the '
                           'hostages and bring the bad guys down.',
               'popularity': 20.283,
               'poster_path': '/fiXHtnQpQiGVkMtyYnEfQ6Gtfx8.jpg',
               'release_date': '1988-07-15',
               'title': 'Die Hard',
               'video': False,
               'vote_average': 7.7,
               'vote_count': 6780}],
  'total_pages': 500,

In [4]:
movie_df = pd.io.json.json_normalize(all_results, record_path = 'results')
movie_df.head()

,popularity,vote_count,video,poster_path,id,adult,backdrop_path,original_language,original_title,genre_ids,title,vote_average,overview,release_date
0,469.061,1779,False,/xBHvZcjRiWyobQ9kxBhO6B2dtRI.jpg,419704,False,/p3TCqUDoVsrIm8fHK9KOTfWnDjZ.jpg,en,Ad Astra,"[12, 18, 9648, 878, 53]",Ad Astra,6.0,"The near future, a time when both hope and har...",2019-09-17
1,250.037,2258,False,/db32LaOibwEliAmSL2jjDF6oDdj.jpg,181812,False,/jOzrELAzFxtMx2I4uDGHOotdfsS.jpg,en,Star Wars: The Rise of Skywalker,"[28, 12, 878]",Star Wars: The Rise of Skywalker,6.6,The surviving Resistance faces the First Order...,2019-12-18
2,214.813,7650,False,/udDclJoHjfjb8Ekgsd4FDteOkCU.jpg,475557,False,/n6bUvigpRFqSwmPp1m2YADdbRBc.jpg,en,Joker,"[80, 18, 53]",Joker,8.3,"During the 1980s, a failed stand-up comedian i...",2019-10-02
3,214.738,978,False,/jyw8VKYEiM1UDzPB7NsisUgBeJ8.jpg,512200,False,/zTxHf9iIOCqRbxvl8W5QYKrsMLq.jpg,en,Jumanji: The Next Level,"[28, 12, 35, 14]",Jumanji: The Next Level,6.7,As the gang return to Jumanji to rescue one of...,2019-12-04
4,157.802,1345,False,/vloNTScJ3w7jwNwtNGoG8DbTThv.jpg,420809,False,/skvI4rYFrKXS73BJxWGH54Omlvv.jpg,en,Maleficent: Mistress of Evil,"[12, 14, 10751]",Maleficent: Mistress of Evil,7.2,Maleficent and her goddaughter Aurora begin to...,2019-10-16


In [5]:
#create CSV file of raw data from OpenWeather API and store in Results Directory

In [7]:
#Create directory to store results
script_dir = os.path.dirname("")
results_dir = os.path.join(script_dir, 'Results/')
if not os.path.isdir(results_dir):
    os.makedirs(results_dir)
movie_df.to_csv(results_dir + output_data_file)

In [8]:
movie_ids = movie_df['id'].tolist()

In [9]:
print(movie_ids[:50])

[419704, 181812, 475557, 512200, 420809, 449924, 330457, 516700, 453405, 466272, 443791, 496243, 593402, 486589, 530915, 181808, 338967, 546554, 503919, 359724, 423204, 461130, 481084, 299536, 431693, 7451, 474350, 539892, 509967, 429617, 398978, 11, 384018, 331482, 522938, 551332, 567609, 578189, 420817, 634904, 920, 245891, 122917, 465086, 10020, 448119, 575094, 525661, 102651, 291805]


In [10]:
def get_movie_revenue(id, api_key):
    
    url = 'https://api.themoviedb.org/3/movie/'

    r = requests.get(url + str(id) + '?api_key='+ api_key+'&language=en-US' + '&sort_by=revenue.desc')
    
    if r.status_code == 200:
        return r.json()
    else:
        return None
    

movie_revenue = []

for id in movie_ids:
    result = get_movie_revenue(id, api_key)
    if result:
        movie_revenue.append(result)


In [14]:
import pprint
pp = pprint.PrettyPrinter()
pp.pprint(movie_revenue[:20])


[{'adult': False,
  'backdrop_path': '/p3TCqUDoVsrIm8fHK9KOTfWnDjZ.jpg',
  'belongs_to_collection': None,
  'budget': 87500000,
  'genres': [{'id': 878, 'name': 'Science Fiction'},
             {'id': 18, 'name': 'Drama'},
             {'id': 53, 'name': 'Thriller'},
             {'id': 12, 'name': 'Adventure'},
             {'id': 9648, 'name': 'Mystery'}],
  'homepage': 'https://www.foxmovies.com/movies/ad-astra',
  'id': 419704,
  'imdb_id': 'tt2935510',
  'original_language': 'en',
  'original_title': 'Ad Astra',
  'overview': 'The near future, a time when both hope and hardships drive '
              'humanity to look to the stars and beyond. While a mysterious '
              'phenomenon menaces to destroy life on planet Earth, astronaut '
              'Roy McBride undertakes a mission across the immensity of space '
              'and its many perils to uncover the truth about a lost '
              'expedition that decades before boldly faced emptiness and '
              'sil

In [22]:
movie_revenue_df = pd.io.json.json_normalize(movie_revenue)
movie_revenue_df.head()

,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,...,status,tagline,title,video,vote_average,vote_count,belongs_to_collection.id,belongs_to_collection.name,belongs_to_collection.poster_path,belongs_to_collection.backdrop_path
0,False,/p3TCqUDoVsrIm8fHK9KOTfWnDjZ.jpg,NaN,87500000,"[{'id': 878, 'name': 'Science Fiction'}, {'id'...",https://www.foxmovies.com/movies/ad-astra,419704,tt2935510,en,Ad Astra,...,Released,The answers we seek are just outside our reach,Ad Astra,False,6.0,1787,NaN,NaN,NaN,NaN
1,False,/jOzrELAzFxtMx2I4uDGHOotdfsS.jpg,NaN,250000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",https://www.starwars.com/films/star-wars-episo...,181812,tt2527338,en,Star Wars: The Rise of Skywalker,...,Released,Every generation has a legend,Star Wars: The Rise of Skywalker,False,6.6,2280,10.0,Star Wars Collection,/iTQHKziZy9pAAY4hHEDCGPaOvFC.jpg,/d8duYyyC9J5T825Hg7grmaabfxQ.jpg
2,False,/n6bUvigpRFqSwmPp1m2YADdbRBc.jpg,NaN,55000000,"[{'id': 80, 'name': 'Crime'}, {'id': 53, 'name...",http://www.jokermovie.net/,475557,tt7286456,en,Joker,...,Released,Put on a happy face.,Joker,False,8.3,7667,NaN,NaN,NaN,NaN
3,False,/zTxHf9iIOCqRbxvl8W5QYKrsMLq.jpg,NaN,125000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://jumanjimovie.com,512200,tt7975244,en,Jumanji: The Next Level,...,Released,,Jumanji: The Next Level,False,6.7,990,495527.0,Jumanji Collection,/6sjMsBcIuqU44GpG5tL33KUFOQR.jpg,/9PCsWrw1GvrZkrd1GCxRqscgZu0.jpg
4,False,/skvI4rYFrKXS73BJxWGH54Omlvv.jpg,NaN,85000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 10751, ...",https://movies.disney.com/maleficent-mistress-...,420809,tt4777008,en,Maleficent: Mistress of Evil,...,Released,Go beyond the fairy tale,Maleficent: Mistress of Evil,False,7.2,1353,531331.0,Maleficent Collection,/eRV721PiB9t8KSyCiv3pnPVioOS.jpg,/fbYvU76vg6xzN24O5ks18Kl3HVs.jpg


In [25]:
movie_revenue_df.to_csv(results_dir + movie_revenue_data)

In [27]:
movie_genres_df = pd.io.json.json_normalize(movie_revenue, "genres",['title'])
movie_genres_df.head(5)

,id,name,title
0,878,Science Fiction,Ad Astra
1,18,Drama,Ad Astra
2,53,Thriller,Ad Astra
3,12,Adventure,Ad Astra
4,9648,Mystery,Ad Astra


In [11]:
# Create genres datafile and save result

In [28]:
movie_genres_df.to_csv(results_dir + movie_genres_data)